### Imports

In [1]:
import main_var
env = "test_new/"
mv = main_var.main_var(env=env)

from article_scraping_lib import *
from article_parsing_lib import *
from openai_module_lib import *
from embedding_keyword_module_lib import *
from utils_art import *

IMPORT : main_var 
IMPORT : utils_art
IMPORT : text_analysis



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


IMPORT : embedding_keyword_module_lib
IMPORT : article_scraping_lib


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


IMPORT : article_parsing_lib
IMPORT : openai_module_lib


## Querring and saving artilce list

In [ ]:
df = loop_scraping(8,'2010-01-01','2024-01-01',sampling_1=12,sampling_2=3,save_steps=True,save_final=True,display=True)

#### Display Statistics

In [ ]:
df = openDFcsv(mv.query_path,"query_fileF")
plotDFstatisticsQuerry(df,70,True)
displayStats(df) 

#### Filter articles with a source that is too/not enought scrapped and source/category/date

In [ ]:
df2 = filterQuerryDataset(df,400,30,True,False,True)
plotDFstatisticsQuerry(df2)
displayStats(df2)

In [ ]:
categroy_l = ["WORLD","NATION","BUSINESS","TECHNOLOGY"]
categroy_l = []
# source_l = ["The New York Times","The Nation","Business Insider","Business Insider"]
source_l = ["Newsweek","Yahoo Finance","ESPN","Fox News"]
source_l = ["World Health Organization (WHO)","Pew Research Center","The New York Times","Nature.com","Tax Foundation","Union of Concerned Scientists"]
df3 = selectOnDf(df2,date_start="2010-01-01", date_end="2024-01-01", categroy_list=categroy_l, source_list=source_l)
displayStats(df3)
saveDFcsv(df3,mv.query_path,"query_fileG_cap")

In [ ]:
df = readArticleFileTable(index_from=0,index_to=11064,save_articles=True,save_final=True,save_steps=True,display_df=True,step_pct=0.01,add_nlp=2,filtered_input_df=True)

In [ ]:
df = joinQuerryAndParse(save=True,remove_invalid=True,display=True,filtered_input_df=True)

In [ ]:
df2 = mainGeneration(True,False,10,9642,7500,9999,True,True,True,0.1)

In [ ]:
emb_mat = extractEmbeddingFromFile(999999999)

In [ ]:
# df = openDFcsv(mv.join1_path,mv.join1_filename)
# genrateKeywordExtract(df, 9641,500,"keywords_list")
df = mainKhttp://localhost:8888/notebooks/OneDrive/Desktop/Article_LLM/main_codebase/MAIN_CODE.ipynb#eywordWF(9641,1000)

In [ ]:
mainJoin()

In [ ]:
col_list = ["category","source_title","year_month","url_TLD","year"]
df_list_len = calculateStatsColList(df,col_list,"len",display_df=True)
df_list_npl = calculateStatsColList(df,col_list,"nlp",display_df=True,display_stats=False) 

In [ ]:
print(df2.dtypes)

In [ ]:
mainJoin()
 

In [2]:
# df = openDFcsv(mv.query_path,mv.query_filename)
# df3 = selectOnDf(df2,"","")
# plotDFstatisticsQuerry(df,70)
# displayStats(df)
# saveDFcsv(df3,mv.query_path,mv.query_filename)



# df = loop_scraping(9,'1999-03-27','2024-03-19',sampling_1=5,sampling_2=5,save_steps=True,save_final=True,display=True)
# plotDFstatisticsQuerry(df,70)
# displayStats(df)

# df = openDFcsv(mv.query_path,mv.query_filename)
# df2 =  filterQuerryDataset(df,150,10,True,True,True)
# plotDFstatisticsQuerry(df2,70,True)
# displayStats(df2)

# df = openDFcsv(mv.query_path,mv.query_filename)
# df_querry = readArticleFileTable(index_from=0,index_to=99999999999,save_articles=True,save_final=True,save_steps=True,display_df=True,step_pct=0.005,add_nlp=2,filtered_input_df=True)
# df_join = joinQuerryAndParse(save=True,remove_invalid=True,display=True,filtered_input_df=True)# df_embd = mainGeneration(True,False,100,99999999,7500,9999,True,True,True,0.005)
# emb_mat = extractEmbeddingFromFile(9999999999)
# df_key = mainKeywordWF(99999999999999,1000)
# df_final = mainJoinOut()

QUERRY dataset loaded from  C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/1_1_query_main/test_new/
QUERRY dataset has entry length of : 11520 

PARSSING dataset loaded from  C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/1_2_scarp_main/test_new/
PARSSING dataset has entry length of : 11520  (100.0% of querry data)

JOINED dataset has entry length of : 11262  (97.76% of parssing data)
JOINED dataset VALID entries : 10125  (89.9% of joined data)
JOINED dataset INVALID entries : 1137  (-10.1% of joined data)

TOTAL yield : from 11520  to  10125 (87.89% yeald)

JOINED dataset saved here : C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/1_4_join_main/test_new/join1_file.csv


In [ ]:
# df = openDFcsv(mv.query_path,mv.query_filename)
# df3 = selectOnDf(df2,"","")
# plotDFstatisticsQuerry(df,70)
# displayStats(df)
# saveDFcsv(df3,mv.query_path,mv.query_filename)


max_num = 10000000
step_pct=0.005
df = loop_scraping(9,'2000-01-01','2024-01-01',sampling_1=5,sampling_2=5,save_steps=True,save_final=True,display=True)
df2 =  filterQuerryDataset(df,150,10,True,True,True)
df_querry = readArticleFileTable(index_from=0,index_to=max_num,save_articles=True,save_final=True,save_steps=True,display_df=True,step_pct=step_pct,add_nlp=2,filtered_input_df=True)
df_join = joinQuerryAndParse(save=True,remove_invalid=True,display=True,filtered_input_df=True)
df_embd = mainGeneration(True,False,10,max_num,7500,1000,True,True,True,step_pct)
emb_mat = extractEmbeddingFromFile(max_num)
df_key = mainKeywordWF(max_num,100)
df_final = mainJoinOut()

In [4]:
print(df_join.dtypes)

title_q           object
link              object
published         object
source_url        object
source_title      object
category          object
year               int64
year_month        object
pk                object
url_list          object
url_TLD           object
url               object
pk_p              object
title_p           object
authors           object
publish_date     float64
keywords_list     object
text_len         float64
valid               bool
tb.sent          float64
tb.noun          float64
tb.word          float64
tb.char          float64
tb.pol           float64
tb.sub           float64
tb.polaj         float64
tb.pos           float64
tb.neg           float64
vs.pos           float64
vs.neu           float64
vs.neg           float64
vs.comp          float64
ts.pos           float64
ts.neg           float64
nlp_error        float64
al.pos           float64
al.neg           float64
dtype: object


In [5]:
col_list = ["category","source_title","year_month","url_TLD","year"]
df_list = calculateStatsColList(df_join,col_list,"len",display_df=True)
df_list = calculateStatsColList(df_join,col_list,"nlp",display_df=True,display_stats=False) #,out_raw=False

KeyError: "['sentence_n', 'noun_n', 'words_n'] not in index"